In [27]:
import pandas as pd
import geopandas as gpd

# Dataset overview

## Geolife

In [2]:
# geolife
df = pd.read_csv("data/processed/geolife.csv")
tessellation = gpd.read_file("data/processed/geolife_tessellation.gpkg")

In [3]:
print("unique users")
df.uid.nunique()

unique users


182

In [4]:
print("unique trips")
df.groupby(["uid", "tid"]).nunique().count()[0]

unique trips


18670

In [5]:
8451/18670

0.45265131226566685

In [6]:
temp = df.groupby(["uid", "tid"]).count().groupby("uid").count()
temp.reset_index()[temp.lat > 400].count()

uid         9
lat         9
lng         9
-           9
Alt         9
dayNo       9
date        9
time        9
datetime    9
dtype: int64

In [7]:
print("distribution of trips per user")
df.groupby(["uid", "tid"]).count().groupby("uid").count().describe()

distribution of trips per user


,lat,lng,-,Alt,dayNo,date,time,datetime
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,102.582418,102.582418,102.582418,102.582418,102.582418,102.582418,102.582418,102.582418
std,250.063575,250.063575,250.063575,250.063575,250.063575,250.063575,250.063575,250.063575
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,27.500000,27.500000,27.500000,27.500000,27.500000,27.500000,27.500000,27.500000
75%,89.750000,89.750000,89.750000,89.750000,89.750000,89.750000,89.750000,89.750000
max,2153.000000,2153.000000,2153.000000,2153.000000,2153.000000,2153.000000,2153.000000,2153.000000


In [8]:
print("tile count and average tile area in square km")
(tessellation.to_crs(21459).area / 1000000).describe()

tile count and average tile area in square km


count    962.000000
mean       3.707657
std        0.017072
min        3.668081
25%        3.695164
50%        3.707726
75%        3.720154
max        3.746816
dtype: float64

In [9]:
print("total tile area in square km")
tessellation.to_crs(21459).area.sum() / 1000000

total tile area in square km


3566.7659211465716

## Berlin

In [76]:
df = pd.read_csv("data/processed/berlin.csv", dtype = {'tile_id': str})
tessellation = gpd.read_file("data/processed/berlin_tessellation.gpkg")

In [33]:
print("unique users")
df.uid.nunique()

unique users


378759

In [34]:
print("unique trips")
df.groupby(["uid", "tid"]).nunique().count()[0]

unique trips


1417134

In [35]:
print("distribution of trips per user")
df.groupby(["uid", "tid"]).count().groupby("uid").count().describe()

distribution of trips per user


,lng,lat,datetime,traffic_mode
count,378759.000000,378759.000000,378759.000000,378759.000000
mean,3.741519,3.741519,3.741519,3.741519
std,1.802168,1.802168,1.802168,1.802168
min,2.000000,2.000000,2.000000,2.000000
25%,2.000000,2.000000,2.000000,2.000000
50%,4.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000
max,16.000000,16.000000,16.000000,16.000000


In [14]:
print("tile count and average tile area in square km")
(tessellation.to_crs(3035).area / 1000000).describe()

tile count and average tile area in square km


count    386.000000
mean       2.308543
std        2.529603
min        0.115578
25%        0.899135
50%        1.555234
75%        2.721739
max       18.449806
dtype: float64

In [15]:
print("total tile area in square km")
tessellation.to_crs(3035).area.sum() / 1000000

total tile area in square km


891.097590393578

In [36]:
df.head()

,uid,tid,lng,lat,datetime,traffic_mode
0,101881652,0,13.349786,52.417182,2018-04-19 05:41:00,PT
1,101881652,0,13.284507,52.490389,2018-04-19 06:46:00,PT
2,101881652,1,13.349786,52.417182,2018-04-19 17:06:00,PT
3,101881652,1,13.284507,52.490389,2018-04-19 16:01:00,PT
4,101881652,2,13.349786,52.417182,2018-04-19 17:37:00,WALK


In [37]:
tessellation.head()

,tile_id,tile_name,geometry
0,110400110,Wiebestraße / Huttenstraße,"POLYGON ((13.32467 52.53087, 13.32465 52.53214..."
1,110500120,Westhafen,"POLYGON ((13.31751 52.53238, 13.31788 52.53243..."
2,110400130,Quitzowstraße / Lehrter Straße,"POLYGON ((13.34402 52.53649, 13.34359 52.53644..."
3,110400140,Heidestraße / Sellerstraße,"POLYGON ((13.37057 52.52706, 13.37095 52.52707..."
4,110400210,Beusselstraße / Turmstraße,"POLYGON ((13.32871 52.53308, 13.32868 52.53281..."


In [39]:
## add tile ID
from shapely.geometry import Point
def assign_points_to_tessellation(df, tessellation):
    # TODO: add progress bar
    gdf = gpd.GeoDataFrame(
        df, geometry=[Point(xy) for xy in zip(df.lng, df.lat)], crs="EPSG:4326"
    )

    # this take some time
    df = gpd.sjoin(
        tessellation[["tile_id", "tile_name", "geometry"]], gdf, how="right"
    )  # Spatial join Points to polygons
    df.drop(["index_left", "geometry"], axis=1, inplace=True)
    return pd.DataFrame(df)

df = assign_points_to_tessellation(df, tessellation)

In [41]:
df.to_csv("data/processed/" + "berlin_w_tile_id" + ".csv", index=False)


## Madrid

In [72]:
df["tid_new"] = df.sort_values("datetime").groupby(["uid", "tid"]).ngroup()


In [59]:
df["point_type"] = "start"
df.sort_values("datetime", inplace=True)
df.loc[df.groupby("tid")["point_type"].tail(1).index, "point_type"] = "end"

In [69]:
order = df[df.point_type == "start"].sort_values(["uid", "datetime"]).tid_new


In [70]:
order

31684         0
31685         1
31686         2
31687         3
31688         4
          ...  
31679    222739
31680    222740
31681    222741
31682    222742
31683    222743
Name: tid_new, Length: 222744, dtype: int64

In [71]:
df.sort_values("tid_new", key = order)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [60]:
df.head()

,uid,tid,datetime,tile_id,lat,lng,tid_new,point_type
52073,1734079_1,1734079_1_1,2018-02-08 09:00:00,079-21-576,40.474285,-3.591467,22925,start
52075,1734079_2,1734079_2_1,2018-02-08 09:00:00,079-21-576,40.474285,-3.591467,22927,start
274819,1734079_2,1734079_2_1,2018-02-08 09:20:00,079-05-101,40.441475,-3.682379,22927,end
274817,1734079_1,1734079_1_1,2018-02-08 09:40:00,049-005,40.428505,-3.560784,22925,end
52079,1734079_4,1734079_4_1,2018-02-08 09:50:00,079-21-576,40.474285,-3.591467,22931,start


In [57]:
df.sort_values("tid_new")

,uid,tid,datetime,tile_id,lat,lng,tid_new
31684,1000154_1,1000154_1_1,2018-05-24 16:30:00,079-16-457,40.459734,-3.649633,0
254428,1000154_1,1000154_1_1,2018-05-24 16:45:00,079-16-463,40.452502,-3.615535,0
31685,1000154_1,1000154_1_2,2018-05-24 22:00:00,079-16-463,40.452502,-3.615535,1
254429,1000154_1,1000154_1_2,2018-05-24 22:30:00,079-16-457,40.459734,-3.649633,1
254430,1000154_2,1000154_2_1,2018-05-24 18:30:00,079-01-014,40.417997,-3.701952,2
...,...,...,...,...,...,...,...
31681,999698_3,999698_3_2,2018-04-26 19:30:00,006-013B,40.530172,-3.639800,222741
254426,999766_1,999766_1_1,2018-04-23 09:00:00,079-16-458,40.461998,-3.642939,222742
31682,999766_1,999766_1_1,2018-04-23 08:30:00,079-15-419,40.434878,-3.638952,222742
31683,999766_1,999766_1_2,2018-04-23 13:30:00,079-16-458,40.461998,-3.642939,222743


In [52]:
df = pd.read_csv("data/processed/madrid.csv")
tessellation = gpd.read_file("data/processed/madrid_tessellation.gpkg")

In [17]:
print("unique users")
df.uid.nunique()

unique users


75208

In [18]:
print("unique trips")
df.groupby(["uid", "tid"]).nunique().count()[0]

unique trips


222744

In [19]:
print("distribution of trips per user")
df.groupby(["uid", "tid"]).count().groupby("uid").count().describe()

distribution of trips per user


,datetime,tile_id,lat,lng
count,75208.000000,75208.000000,75208.000000,75208.000000
mean,2.961706,2.961706,2.961706,2.961706
std,1.498732,1.498732,1.498732,1.498732
min,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,2.000000,2.000000
75%,4.000000,4.000000,4.000000,4.000000
max,20.000000,20.000000,20.000000,20.000000


In [20]:
print("tile count and average tile area in square km")
(tessellation.to_crs(3035).area / 1000000).describe()

tile count and average tile area in square km


count    1259.000000
mean        6.378799
std        16.252493
min         0.011397
25%         0.271355
50%         0.520647
75%         2.654677
max       187.911550
dtype: float64

In [21]:
print("total tile area in square km")
tessellation.to_crs(3035).area.sum() / 1000000

total tile area in square km


8030.908156598319

In [22]:
print("outliers (no tile in tessellation)")
df.lng.isna().value_counts()

False    443481
True       2007
Name: lng, dtype: int64

In [31]:
df[df.uid == "14792_1"].sort_values("tid")

,uid,tid,datetime,tile_id,lat,lng
438,14792_1,14792_1_1,2018-05-08 16:20:00,005-018,40.486413,-3.367882
223182,14792_1,14792_1_1,2018-05-08 16:50:00,19-230-000,NaN,NaN
439,14792_1,14792_1_2,2018-05-09 23:55:00,19-230-000,NaN,NaN
223183,14792_1,14792_1_2,2018-05-08 01:00:00,005-018,40.486413,-3.367882
